<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Importiamo il dataset e rimuoviamo alcune colonne superflue: id dei giocatori,
#nome dell'apertura (visto che abbiamo già il codice), id della partita e le mosse giocate

df=pd.read_csv('games.csv')
df=df.drop(columns=['white_id','black_id','opening_name','moves','id'])

#Ora convertiamo le colonne di inizio e fine partita, che sono in formato Unix time, in formato datetime
#Dopodichè inseriremo una più semplice colonna di durata partita al posto delle due colonne iniziali

df_times = df[['created_at','last_move_at']].copy()

df_times['created_at_dt'] = pd.to_datetime(df_times['created_at']/1000, unit='s', origin='unix')

df_times['last_move_at_dt'] = pd.to_datetime(df_times['last_move_at']/1000, unit='s', origin='unix')

df_times['created_time'] = df_times['created_at_dt'].dt.time
df_times['last_move_time'] = df_times['last_move_at_dt'].dt.time
df_times['duration_seconds'] = (df_times['last_move_at_dt'] - df_times['created_at_dt']).dt.total_seconds()

#Adesso che abbiamo la durata della partita in secondi, inseriamo la colonna nel dataframe e rimuoviamo quelle iniziali

df=df.drop(columns=['created_at','last_move_at'])
df=df.join(df_times['duration_seconds'])

#Ora invece ci occupiamo della colonna 'increment_code', che è della forma "X+Y"
#dove X sono i minuti concessi a ciascun giocatore per giocare le proprie mosse e Y
#è il tempo che gli viene riaggiunto una volta giocata la mossa.
#Vogliamo separare questi due valori e tenerli in colonne separate

#Split di increment_code in due colonne
df[['base_time', 'increment_time']] = (
    df['increment_code'].str.split('+', expand=True).astype(int))

#Rimuoviamo la colonna originale
df = df.drop(columns=['increment_code'])

df.head()
df.tail()

,rated,turns,victory_status,winner,white_rating,black_rating,opening_eco,opening_ply,duration_seconds,base_time,increment_time
20053,True,24,resign,white,1691,1220,A80,2,321.734,10,10
20054,True,82,mate,black,1233,1196,A41,2,744.219,10,0
20055,True,35,mate,white,1219,1286,D00,3,172.834,10,0
20056,True,109,resign,white,1360,1227,B07,4,946.699,10,0
20057,True,78,mate,black,1235,1339,D00,3,736.699,10,0


In [30]:
#Adesso ci occumpiamo di creare dummy per le colonne con valori categoriali:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# fit sull'intera colonna
encoded = enc.fit_transform(df[['victory_status']])

# sostituisci la colonna con il vettore
df['victory_status_oh'] = list(encoded)

colonne_categoriali=['victory_status','opening_eco']
# opzionale: rimuovi la colonna originale
df = df.drop(columns=['victory_status'])
df.head()

,rated,turns,winner,white_rating,black_rating,opening_eco,opening_ply,duration_seconds,base_time,increment_time,victory_status_oh
0,False,13,white,1500,1191,D10,5,0.0,15,2,"[0.0, 0.0, 1.0, 0.0]"
1,True,16,black,1322,1261,B00,4,0.0,5,10,"[0.0, 0.0, 0.0, 1.0]"
2,True,61,white,1496,1500,C20,3,0.0,5,10,"[0.0, 1.0, 0.0, 0.0]"
3,True,61,white,1439,1454,D02,3,0.0,20,0,"[0.0, 1.0, 0.0, 0.0]"
4,True,95,white,1523,1469,C41,5,0.0,30,3,"[0.0, 1.0, 0.0, 0.0]"
